In [ ]:
import pandas as pd
from collections import Counter
import random
df = pd.read_json("ScaleQuest-Math.json", lines=True)

In [ ]:
import random
def generate_non_consecutive_numbers(X, n):
    max_possible = (X + 1) // 2 + (X % 2)
    if n > max_possible:
        raise ValueError(f"Cannot generate {n} non-consecutive numbers in range [0, {X}]")
    while True:
        candidates = list(range(X + 1))
        random.shuffle(candidates)
        result = []
        banned = set()
        for num in candidates:
            if num in banned:
                continue
            result.append(num)
            banned.update({num - 1, num + 1})
            if len(result) == n:
                return sorted(result)
def data_process(idx):
    data = df.iloc[idx]['response'].split("\n\n")
    # 20% num_to_remove = 0
    if random.random() < 0.2:
        num_to_remove = 0
    else:
        if len(data) <= 10:
            num_to_remove = random.randint(1, 2)  # remove 1-2 steps
        else:
            num_to_remove = random.randint(1, 3)  # remove 1-3 steps
    
    removed_steps = []
    removed_items = []
    if num_to_remove > 0:
        removed_steps = generate_non_consecutive_numbers(len(data) - 3, num_to_remove)
    remain_data = [data[i] for i in range(len(data)) if i not in removed_steps]
    removed_items = [data[i] for i in range(len(data)) if i in removed_steps]
    result = "\n".join([f"step{i+1}:\n{item}" for i, item in enumerate(remain_data)])
    
    sy = """You are a mathematics teacher reviewing a solution that may be missing one or more steps.
Your task is to:
1. Identify all points in the logical flow where a step is missing. For each missing step, specify exactly between which two consecutive steps it should be placed.
2. Provide the complete missing step(s) with necessary explanations and equations.
The solution may be missing multiple steps or might be complete. The steps in the solution are labeled from Step 0 (problem statement) to Step N.
Please format your response as follows:
For each missing step, output:
Missing Step X:
The missing step should be placed between Step Y and Step Y+1.
The missing step is:
[Write the complete missing step here with necessary explanations and equations]
If there are no missing steps, please output:
No missing steps.
"""
    
    prompt = f"""<incomplete_solution>
Step 0:
{df.iloc[idx]['query']}
{result}
</incomplete_solution>
"""
    assist = ""
    if removed_steps:
        for i, (removed_step, removed_item) in enumerate(zip(removed_steps, removed_items), start=1):
            assist += f"Missing Step {i}：\nThe missing step should be placed between Step {removed_step-i+1} and Step {removed_step-i+2}.\n"
            assist += f"The missing step is:\n{removed_item}\n"
    else:
        assist = "No missing steps."

    m = {
        "messages": [
          {
            "role": "system",
            "content": sy
          },
          {
            "role": "user",
            "content": prompt
          },
          {
            "role": "assistant",
            "content": assist
          }
        ]
      }
    return m

In [ ]:
import json
from tqdm import tqdm

process_data = []

for i in tqdm(range(len(df))):
    data = df.iloc[i]['response'].split("\n\n")
    if len(data) >= 6:
        process_data.append(data_process(i))

with open('ScaleQM+_total.json', 'w') as f:
    json.dump(process_data, f, ensure_ascii=False, indent=4)

print("process_data has been saved to JSON files.")